# MORO playground - round 2

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ema_workbench import (MultiprocessingEvaluator, 
                           Scenario, Constraint, Policy,
                           ScalarOutcome)
from ema_workbench.em_framework.optimization import EpsilonProgress, HyperVolume
from ema_workbench.em_framework.evaluators import (perform_experiments, optimize,
                                                   BaseEvaluator)
from ema_workbench.util import ema_logging, save_results, load_results
from ema_workbench.analysis import (pairs_plotting, prim, 
                                    feature_scoring, parcoords,
                                    dimensional_stacking)

from problem_formulation import get_model_for_problem_formulation
%matplotlib inline

ema_logging.log_to_stderr(ema_logging.INFO)

BaseEvaluator.reporting_frequency = 0.01

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [2]:
'''
For different list of outcomes:
# 0 = 2-objective PF
# 1 = 3-objective PF
# 2 = 5-objective PF
# 3 = Disaggregate over locations
# 4 = Disaggregate over time
# 5 = Fully disaggregated
'''
dike_model, planning_steps = get_model_for_problem_formulation(3) # assign problem_formulation_id

## Load the levers found from MORO

In [3]:
results = load_results('./results/robust_opt_500nfe_200scenario.tar.gz')

experiments, _ = results

[MainProcess/INFO] results loaded succesfully from D:\EPA\EPA Semester-1.4\EPA1361 Model-based Decision-making\MBDM-group-11\final assignment\results\robust_opt_500nfe_200scenario.tar.gz


In [4]:
# check the found solutions
levers = experiments.loc[:, [l.name for l in dike_model.levers]]
levers.T

,0,1,2,3,4,5,6,7
0_RfR 0,0,1,0,1,1,0,0,0
0_RfR 1,0,1,0,1,0,0,1,1
0_RfR 2,1,1,1,1,0,0,1,1
1_RfR 0,1,0,0,0,0,0,0,0
1_RfR 1,0,0,0,1,0,0,0,0
1_RfR 2,0,1,0,0,0,0,0,0
2_RfR 0,1,1,1,0,0,0,1,0
2_RfR 1,0,1,1,1,1,1,0,1
2_RfR 2,1,1,0,0,0,1,0,1
3_RfR 0,0,0,1,1,0,0,1,1


In [5]:
policies_to_evaluate = [Policy('policy'+str(i), **policy.to_dict()) 
                        for i, policy in pd.DataFrame.from_dict(levers).iterrows()]

In [6]:
n_scenarios = 1000

In [7]:
%%time
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios= n_scenarios,
                                            policies = policies_to_evaluate)

save_results(results, './results/MOROsolutions_disaggregatedOverLocation.tar.gz')

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 1000 scenarios * 8 policies * 1 model(s) = 8000 experiments
[MainProcess/INFO] 800 cases completed
[MainProcess/INFO] 1600 cases completed
[MainProcess/INFO] 2400 cases completed
[MainProcess/INFO] 3200 cases completed
[MainProcess/INFO] 4000 cases completed
[MainProcess/INFO] 4800 cases completed
[MainProcess/INFO] 5600 cases completed
[MainProcess/INFO] 6400 cases completed
[MainProcess/INFO] 7200 cases completed
[MainProcess/INFO] 8000 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool
[MainProcess/INFO] results saved successfully to D:\EPA\EPA Semester-1.4\EPA1361 Model-based Decision-making\MBDM-group-11\final assignment\results\MOROsolutions_disaggregatedOverLocation.tar.gz


Wall time: 1h 7min 31s
